***Vorlesung 'Syntax natürlicher Sprachen'***

--- 
# Vorlesung 2: Einführung Parsing 

- **Parsing: algorithmisches Verfahren zur Verarbeitung formaler Grammatik**
- **Ziel: Finden einer Ableitung aus den Regeln der Grammatik zu einem Input-Satz** (Überprüfung auf Wohlgeformtheit)


- **formale Grammatik: Regeln zur Beschreibung syntaktischer Strukturen**
  - ermöglicht **Generierung** aller wohlgeformten Sätze der dadurch beschriebenen Sprache
  - kompakte Darstellung (durch wiederholte Regelanwendung und Rekursion kann eine unendliche Menge an Sätzen generiert werden)
  - **Konstituentenregeln** (CFG) oder **Dependenzregeln** (*Dependency Grammar*)!


- https://www.nltk.org/book/ch08.html#summary

In [1]:
import nltk

---
## 1. Generierung mit formaler Grammatik

- Ableitung von Sätzen ausgehend von Startsymbol `S` (vgl. top-down-Parser)

---
### Formale Grammatik: CFG

- **CFG  = Kontextfreie Grammatik**
- auch: **Phrasenstrukturgrammatik**
- Konstituentenregeln (syntagmatische Struktur)

> Given a set of syntactic categories, a context-free grammar uses a set of productions to say how a phrase of some category A can be analyzed into a sequence of smaller parts. (https://www.nltk.org/book/ch08.html#summary)


In [2]:
grammar = nltk.CFG.fromstring("""
## Syntaktische Regeln:
    S -> NP VP
    PP -> P NP
    NP -> Det N
    NP -> NP PP
    NP -> Pron
    VP -> V NP | VP PP
##Lexikalische Regeln:
    Pron -> 'I'      
    Det -> 'an' | 'my'
    N -> 'elephant' | 'pajamas'
    V -> 'shot'
    P -> 'in'
""")

In [3]:
#Generierung (depth = maximale Anzahl an Regelanwendungen):
from nltk.parse.generate import generate
for sentence in generate(grammar, depth=5):
    print(' '.join(sentence))

an elephant shot an elephant
an elephant shot an pajamas
an elephant shot my elephant
an elephant shot my pajamas
an elephant shot I
an pajamas shot an elephant
an pajamas shot an pajamas
an pajamas shot my elephant
an pajamas shot my pajamas
an pajamas shot I
my elephant shot an elephant
my elephant shot an pajamas
my elephant shot my elephant
my elephant shot my pajamas
my elephant shot I
my pajamas shot an elephant
my pajamas shot an pajamas
my pajamas shot my elephant
my pajamas shot my pajamas
my pajamas shot I
I shot an elephant
I shot an pajamas
I shot my elephant
I shot my pajamas
I shot I


---
## 2. CFG-Parsing: Ambiguität und Rekursion 


---
### Verwendung CFG-Parser (Beispiel: PP-Attachment-Ambiguität)

- verwendeter Parser: Chart-Parser

In [4]:
sent = 'I shot an elephant in my pajamas'.split()
print(sent)

['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']


In [5]:
parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print(unicodelines=True)

(S
  (NP (Pron I))
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
      S                                       
 ┌────┴──────────────┐                         
 │                   VP                       
 │         ┌─────────┴──────────┐              
 │         VP                   PP            
 │    ┌────┴───┐            ┌───┴───┐          
 NP   │        NP           │       NP        
 │    │    ┌───┴─────┐      │   ┌───┴─────┐    
Pron  V   Det        N      P  Det        N   
 │    │    │         │      │   │         │    
 I   shot  an     elephant  in  my     pajamas

(S
  (NP (Pron I))
  (VP
    (V shot)
    (NP
      (NP (Det an) (N elephant))
      (PP (P in) (NP (Det my) (N pajamas))))))
      S                                       
 ┌────┴──────────────┐                         
 │                   VP                       
 │    ┌──────────────┴──────┐                  
 │    │                     NP                


---
### Koordinationsambiguität

In [6]:
grammar = nltk.CFG.fromstring("""
## linke Seite erster Regel = Startsymbol:
    NP -> NP CON NP
    NP -> Adj NP
    NP -> N
    CON -> 'und'
    Adj -> 'alte'
    N -> 'Männer' | 'Frauen'
""")

sent = 'alte Männer und Frauen'.split()

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print(unicodelines=True)

(NP (NP (Adj alte) (NP (N Männer))) (CON und) (NP (N Frauen)))
           NP             
      ┌────┴─────┬────┐    
      NP         │    │   
 ┌────┴────┐     │    │    
 │         NP    │    NP  
 │         │     │    │    
Adj        N    CON   N   
 │         │     │    │    
alte     Männer und Frauen

(NP (Adj alte) (NP (NP (N Männer)) (CON und) (NP (N Frauen))))
             NP           
 ┌───────────┴───┐         
 │               NP       
 │     ┌─────────┼────┐    
 │     NP        │    NP  
 │     │         │    │    
Adj    N        CON   N   
 │     │         │    │    
alte Männer     und Frauen



---

### Verarbeitungsproblem 1:  Linksrekursion (nicht mit RekursiveDescent-Parser möglich)

- linksrekursive Regeln: `NP -> NP PP`, `VP -> VP PP`

In [7]:
grammar = nltk.CFG.fromstring("""
## Syntaktische Regeln:
    S -> NP VP
    PP -> P NP
    NP -> Det N 
#rekursive Regel:
    NP -> NP PP
    NP -> Pron
    VP -> V NP
#rekursive Regel:    
    VP -> VP PP
##Lexikalische Regeln:
    Pron -> 'I'      
    Det -> 'an' | 'my'
    N -> 'elephant' | 'pajamas'
    V -> 'shot'
    P -> 'in'
""")

In [8]:
sent = 'I shot an elephant'.split()
print(sent)

['I', 'shot', 'an', 'elephant']


#### Recursive-Descent-Parser (einfacher top-down-Parser)
- https://www.nltk.org/book/ch08.html#recursive-descent-parsing

##### *Endlosschleife bei linksrekursiver Regel!*

In [9]:
#parser = nltk.RecursiveDescentParser(grammar,trace=2)
#for tree in parser.parse(sent):
#    print(tree)
#    tree.pretty_print(unicodelines=True)

```
Parsing 'I shot an elephant'
    [ * S ]
  E [ * NP VP ]
  E [ * Det N VP ]
  E [ * 'an' N VP ]
  E [ * 'my' N VP ]
  E [ * NP PP VP ]
  E [ * Det N PP VP ]
  E [ * 'an' N PP VP ]
  E [ * 'my' N PP VP ]
  E [ * NP PP PP VP ]
  E [ * Det N PP PP VP ]
  E [ * 'an' N PP PP VP ]
  E [ * 'my' N PP PP VP ]
  E [ * NP PP PP PP VP ]
  E [ * Det N PP PP PP VP ]
  E [ * 'an' N PP PP PP VP ]
  E [ * 'my' N PP PP PP VP ]
  E [ * NP PP PP PP PP VP ]
  E [ * Det N PP PP PP PP VP ]
  E [ * 'an' N PP PP PP PP VP ]
  E [ * 'my' N PP PP PP PP VP ]
  E [ * NP PP PP PP PP PP VP ]
  E [ * Det N PP PP PP PP PP VP ]
  E [ * 'an' N PP PP PP PP PP VP ]
  E [ * 'my' N PP PP PP PP PP VP ]
  E [ * NP PP PP PP PP PP PP VP ]
  E [ * Det N PP PP PP PP PP PP VP ]
  E [ * 'an' N PP PP PP PP PP PP VP ]
  E [ * 'my' N PP PP PP PP PP PP VP ]
  ...
```


#### Parsing möglich z.B. mit Shift-Reduce-Parser (bottom-up)
- https://www.nltk.org/book/ch08.html#shift-reduce-parsing

In [10]:
parser = nltk.ShiftReduceParser(grammar,trace=2)
for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print()

Parsing 'I shot an elephant'
    [ * I shot an elephant]
  S [ 'I' * shot an elephant]
  R [ Pron * shot an elephant]
  R [ NP * shot an elephant]
  S [ NP 'shot' * an elephant]
  R [ NP V * an elephant]
  S [ NP V 'an' * elephant]
  R [ NP V Det * elephant]
  S [ NP V Det 'elephant' * ]
  R [ NP V Det N * ]
  R [ NP V NP * ]
  R [ NP VP * ]
  R [ S * ]
(S (NP (Pron I)) (VP (V shot) (NP (Det an) (N elephant))))
           S                  
  _________|___                
 |             VP             
 |     ________|___            
 NP   |            NP         
 |    |         ___|_____      
Pron  V       Det        N    
 |    |        |         |     
 I   shot      an     elephant



#### ebenso mit Chart-Parser (bottom-up mit dynamischer Programmierung)
- https://www.nltk.org/book/ch08.html#well-formed-substring-tables

In [11]:
parser = nltk.ChartParser(grammar,trace=1)
for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print()

|.    I    .   shot  .    an   . elephant.|
|[---------]         .         .         .| [0:1] 'I'
|.         [---------]         .         .| [1:2] 'shot'
|.         .         [---------]         .| [2:3] 'an'
|.         .         .         [---------]| [3:4] 'elephant'
|[---------]         .         .         .| [0:1] Pron -> 'I' *
|[---------]         .         .         .| [0:1] NP -> Pron *
|[--------->         .         .         .| [0:1] S  -> NP * VP
|[--------->         .         .         .| [0:1] NP -> NP * PP
|.         [---------]         .         .| [1:2] V  -> 'shot' *
|.         [--------->         .         .| [1:2] VP -> V * NP
|.         .         [---------]         .| [2:3] Det -> 'an' *
|.         .         [--------->         .| [2:3] NP -> Det * N
|.         .         .         [---------]| [3:4] N  -> 'elephant' *
|.         .         [-------------------]| [2:4] NP -> Det N *
|.         .         [------------------->| [2:4] S  -> NP * VP
|.         .         

---
### Verarbeitungsproblem 2: Temporale Ambiguität *(benötigt Backtracking)*

In [12]:
sent = 'the old man the boat'.split()

grammar = nltk.CFG.fromstring("""
## Syntaktische Regeln:
    S -> NP VP
    NP -> Det Adj N
    NP -> Det N
    VP -> V NP
##Lexikalische Regeln:
    Det -> 'the'
    Adj -> 'old'    
    N -> 'man' | 'boat' | 'old'
    V -> 'man'
""")


#### ShiftReduceParser im NLTK: ohne Backtracking! *(findet keinen Parse)*

In [13]:
#NLTK-ShiftReduceParser (bottom-up-Parser): kein Backtracking! 
#bleibt bei Analyse NP NP ((the old man) (the boat)) stehen, findet keinen vollständigen Parse

parser = nltk.ShiftReduceParser(grammar,trace=2)

for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print(unicodelines=True)

Parsing 'the old man the boat'
    [ * the old man the boat]
  S [ 'the' * old man the boat]
  R [ Det * old man the boat]
  S [ Det 'old' * man the boat]
  R [ Det Adj * man the boat]
  S [ Det Adj 'man' * the boat]
  R [ Det Adj N * the boat]
  R [ NP * the boat]
  S [ NP 'the' * boat]
  R [ NP Det * boat]
  S [ NP Det 'boat' * ]
  R [ NP Det N * ]
  R [ NP NP * ]


#### dagegen: RecursiveDescentParser im NLTK: hat Backtracking implementiert:

In [14]:
##NLTK-RecursiveDescentParser (top-down-Parser): Backtracking implementiert 
## Backtracking ermöglicht Reanalyse der Struktur

parser = nltk.RecursiveDescentParser(grammar,trace=3)

for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print(unicodelines=True)
    

Parsing 'the old man the boat'
Start:
    [ * S ]
Expand: S -> NP VP
    [ * NP VP ]
Expand: NP -> Det Adj N
    [ * Det Adj N VP ]
Expand: Det -> 'the'
    [ * 'the' Adj N VP ]
Match: 'the'
    [ 'the' * Adj N VP ]
Expand: Adj -> 'old'
    [ 'the' * 'old' N VP ]
Match: 'old'
    [ 'the' 'old' * N VP ]
Expand: N -> 'man'
    [ 'the' 'old' * 'man' VP ]
Match: 'man'
    [ 'the' 'old' 'man' * VP ]
Expand: VP -> V NP
    [ 'the' 'old' 'man' * V NP ]
Expand: V -> 'man'
    [ 'the' 'old' 'man' * 'man' NP ]
Backtrack: 'the' match failed
Expand: N -> 'boat'
    [ 'the' 'old' * 'boat' VP ]
Backtrack: 'man' match failed
Expand: N -> 'old'
    [ 'the' 'old' * 'old' VP ]
Backtrack: 'man' match failed
Expand: NP -> Det N
    [ * Det N VP ]
Expand: Det -> 'the'
    [ * 'the' N VP ]
Match: 'the'
    [ 'the' * N VP ]
Expand: N -> 'man'
    [ 'the' * 'man' VP ]
Backtrack: 'old' match failed
Expand: N -> 'boat'
    [ 'the' * 'boat' VP ]
Backtrack: 'old' match failed
Expand: N -> 'old'
    [ 'the' * 'old

---

---

## 3. Dependency Parsing

--- 

### Formale Grammatik: Dependenzgrammatik

- formale Grammatik mit Regeln bzgl. Dependenzrelationen zwischen Wörtern (Subjekt, Objekt, Attribut usw.)
- abstrahiert von linearer Anordnung

> A dependency grammar uses productions to specify what the dependents are of a given lexical head. (https://www.nltk.org/book/ch08.html#summary)

- https://www.nltk.org/book/ch08.html#dependencies-and-dependency-grammar

In [15]:
grammar = nltk.DependencyGrammar.fromstring("""
    'beißen' -> 'Hunde' 
    'Hunde' -> 'bellen' 
    'bellen' -> 'die' 
    """)

### Beispiel Relativsatz

In [16]:
sent = 'Hunde    die bellen    beißen'.split()
print(sent)

['Hunde', 'die', 'bellen', 'beißen']


In [17]:
parser = nltk.ProjectiveDependencyParser(grammar)

for tree in parser.parse(sent):
    print(tree, "\n")
    tree.pretty_print(unicodelines=True)

(beißen (Hunde (bellen die))) 

beißen
  │    
Hunde 
  │    
bellen
  │    
 die  



---
### Verarbeitungsproblem 3a (Dependency Parsing): Nicht-projektive Strukturen

- auch: *long distance dependency*, diskontinuierliche Strukturen
    - vgl. https://en.wikipedia.org/wiki/Discontinuity_(linguistics)#Projectivity


- können mit Dependenzgrammatiken modelliert werden (abstrahiert von linearer Anordnung) 
- aber: **nur bestimmte Dependency Parsingalgorithmen erlauben die Verarbeitung solcher Strukturen**


#### Beispiel: ins Nachfeld extrahierter Relativsatz (Rechtsversetzung):

In [18]:
sent = 'Hunde beißen   die bellen '.split()
print(sent)

['Hunde', 'beißen', 'die', 'bellen']


#### ProjectiveDependencyParser: kein Parse!

In [19]:
#kein Parse mit ProjectiveDependencyParser!
parser = nltk.ProjectiveDependencyParser(grammar)

for tree in parser.parse(sent):
    print(tree, "\n")
    tree.pretty_print(unicodelines=True)

#### aber mit NonprojectiveDependencyParser:

In [20]:
#NonprojectiveDependencyParser: erkennt Struktur korrekt
#Code s. http://www.nltk.org/howto/dependency.html:

dp = nltk.NonprojectiveDependencyParser(grammar)
g, = dp.parse(sent)

print(g.root['word'])

for _, node in sorted(g.nodes.items()):
    if node['word'] is not None:
        print('{address} {word}: {d}'.format(d=node['deps'][''], **node))

print('\n', g.tree(), '\n')
g.tree().pretty_print(unicodelines=True)

beißen
1 Hunde: [4]
2 beißen: [1]
3 die: []
4 bellen: [3]

 (beißen (Hunde (bellen die))) 

beißen
  │    
Hunde 
  │    
bellen
  │    
 die  



---

## 4. Nicht-projektive Strukturen in CFGs

- im Gegensatz zu Dependengrammatiken: Beschreibung von nicht-projektiven Strukturen mit CFGs nicht möglich (überkreuzende Kanten)
- Analysierbar z.B. mit Transformationsregeln (Movement, Leerstellen)

https://en.wikipedia.org/wiki/Discontinuity_(linguistics)#Constituency-based_projectivity

### projektiver Relativsatz mit CFG:

In [21]:
#projektiv:
grammar = nltk.CFG.fromstring("""
## Syntaktische Regeln:
    S -> NP VP
    VP -> V
  #Relativsatz:
    NP -> N SREL
    SREL -> RELPRO VP
##Lexikalische Regeln:
    RELPRO -> 'die'
    N -> 'Hunde' 
    V -> 'beißen' | 'bellen' 
""")

parser = nltk.ChartParser(grammar,trace=0)

sent = 'Hunde  die bellen  beißen'.split()

for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)    

              S                
        ┌─────┴────────────┐    
        NP                 │   
  ┌─────┴─────┐            │    
  │          SREL          │   
  │     ┌─────┴─────┐      │    
  │     │           VP     VP  
  │     │           │      │    
  N   RELPRO        V      V   
  │     │           │      │    
Hunde  die        bellen beißen



### Verarbeitungsproblem 3b (CFG): nicht-projektiver Satz (rechtsversetzter Relativsatz)

- nur mit Zusatzregel: `SREL -> `

In [22]:
#nicht-projektiv > Überkreuzung: nur mit Leerstelle analyierbar (Movement VP)
grammar = nltk.CFG.fromstring("""
## Syntaktische Regeln:
    S -> NP VP SREL
    VP -> V
  #Relativsatz:
    NP -> N SREL
    SREL -> RELPRO VP
  #Leerstelle:    
    SREL -> 
##Lexikalische Regeln:
    RELPRO -> 'die'
    N -> 'Hunde' 
    V -> 'beißen' | 'bellen'  
""")

parser = nltk.ChartParser(grammar,trace=0)

sent = 'Hunde beißen   die bellen '.split()

for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)    

                 S                      
       ┌─────────┼────────────┐          
       │         │           SREL       
       │         │      ┌─────┴─────┐    
       NP        VP     │           VP  
  ┌────┴───┐     │      │           │    
  N       SREL   V    RELPRO        V   
  │        │     │      │           │    
Hunde     ...  beißen  die        bellen



## 5. Rekursives *center-embedding* mit CFGs

- **Relativsatz mit transitivem Verb: Beispiel für *center-embedding*-Struktur**
- Argument für CFG als Phrasenstrukturgrammatik (nicht mit regulärer Grammatik möglich)


In [23]:
grammar = nltk.CFG.fromstring("""
## Syntaktische Regeln:
    S -> NP VP
    NP -> N
  #center-embedding Regel (vereinfacht mit direkter Rekursion):  
    NP -> N RELPRO NP V
    VP -> V
##Lexikalische Regeln:
    RELPRO -> 'die'
    N -> 'Hunde' | 'Menschen'
    V -> 'beißen' | 'bellen' | 'lieben' 
""")

parser = nltk.ChartParser(grammar,trace=0)

In [24]:
sent = 'Hunde die Menschen beißen bellen'.split()

for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)    

                S                  
        ┌───────┴──────────────┐    
        NP                     │   
  ┌─────┼───────┬───────┐      │    
  │     │       NP      │      VP  
  │     │       │       │      │    
  N   RELPRO    N       V      V   
  │     │       │       │      │    
Hunde  die   Menschen beißen bellen



In [25]:
sent = 'Hunde die Menschen die Hunde lieben beißen bellen'.split()

for tree in parser.parse(sent):
    tree.pretty_print(unicodelines=True)    

                               S                       
                        ┌──────┴───────────────────┐    
                        NP                         │   
  ┌─────┬───────────────┼───────────────────┐      │    
  │     │               NP                  │      │   
  │     │       ┌───────┼──────┬─────┐      │      │    
  │     │       │       │      NP    │      │      VP  
  │     │       │       │      │     │      │      │    
  N   RELPRO    N     RELPRO   N     V      V      V   
  │     │       │       │      │     │      │      │    
Hunde  die   Menschen  die   Hunde lieben beißen bellen

